# Задача 10. Graph Convolutional Network

- Найти графовый набор данных для решения задачи предсказания (классификация вершин, обнаружение сообществ и т.д.).
- Использовать несколько слоев GCNConv из библиотеки PyG для построения GCN модели.
- Обучить полученную модель, подобрать гиперпараметры (например, learning rate) на валидационной выборке, и оценить качество предсказания на тестовой выборке.
- (+5 баллов) Также представить самостоятельную реализацию слоя GCNConv, используя матричные операции. Повторить обучение с собственными слоями и сравнить результаты.

In [1]:
!pip install torch-scatter torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-2.1.0+cu121.html

Looking in links: https://data.pyg.org/whl/torch-2.1.0+cu121.html


# Датасет

В качестве датасета используется Cora - датасет для задач классификации графов. Признак вершины - слова отражающие содержание статьи.
Задача заключается в определении тематики статьи по её содержанию и связям

In [2]:
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0]


/usr/local/lib/python3.11/dist-packages/torch_geometric/typing.py:86: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: /usr/local/lib/python3.11/dist-packages/torch_scatter/_version_cuda.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
/usr/local/lib/python3.11/dist-packages/torch_geometric/typing.py:97: UserWarning: An issue occurred while importing 'torch-cluster'. Disabling its usage. Stacktrace: /usr/local/lib/python3.11/dist-packages/torch_cluster/_version_cuda.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev
  warnings.warn(f"An issue occurred while importing 'torch-cluster'. "
/usr/local/lib/python3.11/dist-packages/torch_geometric/typing.py:113: UserWarning: An issue occurred while importing 'torch-spline-conv'. Disabling its usage. Stacktrace: /usr/local/lib/python3.11/dist-packages/torch_spline_conv/_version_cuda.so: undefined symbol: _ZN3c1017Registe

In [3]:
print(f"Число признаков на вершину: {dataset.num_features}")
print(f"Число классов: {dataset.num_classes}")
print(f"Число вершин: {data.num_nodes}")
print(f"Число рёбер: {data.num_edges}")
print(f"Доля обучающей выборки: {data.train_mask.sum().item()} узлов")
print(f"Доля валидации: {data.val_mask.sum().item()} узлов")
print(f"Доля тестовой выборки: {data.test_mask.sum().item()} узлов")
print(f"Наличие самосвязей (self-loops): {data.has_self_loops()}")
print(f"Граф ориентированный? {'Да' if data.is_directed() else 'Нет'}")


Число признаков на вершину: 1433
Число классов: 7
Число вершин: 2708
Число рёбер: 10556
Доля обучающей выборки: 140 узлов
Доля валидации: 500 узлов
Доля тестовой выборки: 1000 узлов
Наличие самосвязей (self-loops): False
Граф ориентированный? Нет


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import Planetoid
from torch_geometric.utils import add_self_loops, degree


# GCN

In [5]:
class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, dataset.num_classes)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

model = GCN(hidden_channels=16)

# Кастомный GCN

In [6]:
class CustomGCNConv(nn.Module):
    def __init__(self, in_channels, out_channels, bias=True):
        super().__init__()
        self.weight = nn.Parameter(torch.Tensor(in_channels, out_channels))
        if bias:
            self.bias = nn.Parameter(torch.Tensor(out_channels))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        nn.init.xavier_uniform_(self.weight)
        if self.bias is not None:
            nn.init.zeros_(self.bias)

    def forward(self, x, edge_index):
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))
        row, col = edge_index
        deg = degree(row, x.size(0), dtype=x.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]

        adj = torch.sparse_coo_tensor(
            indices=edge_index,
            values=norm,
            size=(x.size(0), x.size(0)),
            device=x.device
        )

        x = torch.mm(x, self.weight)
        out = torch.sparse.mm(adj, x)

        if self.bias is not None:
            out += self.bias
        return out

In [7]:
class CustomGCN(nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.conv1 = CustomGCNConv(dataset.num_features, hidden_channels)
        self.conv2 = CustomGCNConv(hidden_channels, dataset.num_classes)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = data.to(device)

def create_model(use_custom=False):
    if use_custom:
        return CustomGCN(hidden_channels=16).to(device)
    else:
        from torch_geometric.nn import GCNConv
        class PyGGCN(torch.nn.Module):
            def __init__(self, hidden_channels):
                super().__init__()
                self.conv1 = GCNConv(dataset.num_features, hidden_channels)
                self.conv2 = GCNConv(hidden_channels, dataset.num_classes)
            def forward(self, x, edge_index):
                x = self.conv1(x, edge_index).relu()
                x = F.dropout(x, p=0.5, training=self.training)
                x = self.conv2(x, edge_index)
                return F.log_softmax(x, dim=1)
        return PyGGCN(hidden_channels=16).to(device)

def train(model, optimizer, data):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

def test(model, data):
    model.eval()
    out = model(data.x, data.edge_index)
    pred = out.argmax(dim=1)
    accs = []
    for mask in [data.train_mask, data.val_mask, data.test_mask]:
        accs.append((pred[mask] == data.y[mask]).sum().item() / mask.sum().item())
    return accs

In [9]:
results = {}
for model_name, use_custom in [('PyG GCN', False), ('Custom GCN', True)]:
    model = create_model(use_custom)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

    best_val_acc = 0
    for epoch in range(1, 201):
        loss = train(model, optimizer, data)
        train_acc, val_acc, test_acc = test(model, data)

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        if epoch % 10 == 0:
            print(f'{model_name} | Epoch: {epoch:03d}, Loss: {loss:.4f}, '
                  f'Train: {train_acc:.4f}, Val: {val_acc:.4f}, Test: {test_acc:.4f}')

    results[model_name] = best_test_acc
    print(f'Optimized {model_name} | Best Test Acc: {best_test_acc:.4f}')

print('\nСравнение:')
for name, acc in results.items():
    print(f'{name}: {acc:.4f}')

PyG GCN | Epoch: 010, Loss: 0.9049, Train: 0.9714, Val: 0.7700, Test: 0.7820
PyG GCN | Epoch: 020, Loss: 0.2320, Train: 0.9929, Val: 0.7940, Test: 0.8100
PyG GCN | Epoch: 030, Loss: 0.0749, Train: 1.0000, Val: 0.7740, Test: 0.8010
PyG GCN | Epoch: 040, Loss: 0.0632, Train: 1.0000, Val: 0.7860, Test: 0.8010
PyG GCN | Epoch: 050, Loss: 0.0461, Train: 1.0000, Val: 0.7780, Test: 0.7910
PyG GCN | Epoch: 060, Loss: 0.0705, Train: 1.0000, Val: 0.7800, Test: 0.8020
PyG GCN | Epoch: 070, Loss: 0.0353, Train: 1.0000, Val: 0.7760, Test: 0.7950
PyG GCN | Epoch: 080, Loss: 0.0341, Train: 1.0000, Val: 0.7720, Test: 0.7890
PyG GCN | Epoch: 090, Loss: 0.0510, Train: 1.0000, Val: 0.7760, Test: 0.7990
PyG GCN | Epoch: 100, Loss: 0.0476, Train: 1.0000, Val: 0.7740, Test: 0.7990
PyG GCN | Epoch: 110, Loss: 0.0306, Train: 1.0000, Val: 0.7820, Test: 0.8050
PyG GCN | Epoch: 120, Loss: 0.0411, Train: 1.0000, Val: 0.7780, Test: 0.8090
PyG GCN | Epoch: 130, Loss: 0.0329, Train: 1.0000, Val: 0.7740, Test: 0.8080

# Результат

Для кастомной и базовой реализации точности достигли 0.8080 и 0.8090, соответственно.
Полученные результаты демонстрируют схожую точность на тестовых данных, что говорит о правильности кастомной реализации